### Change the working directory to load the utils

In [ ]:
import os
from os import listdir
import os.path
from os.path import isfile, join
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import math
import re
import scipy as sp

In [ ]:
np.random.seed(10)

# Global configuration

In [ ]:
root_path = "../../"

sel_dataset = "emotion"
print("Main setup: \t " + sel_dataset)

#Load the problem labels
labels_file_path = root_path + "datasets/%s_dataset_train/%s_labels.txt" % (sel_dataset, sel_dataset)
labels = open(labels_file_path, 'r').readlines()
labels = [s.rstrip() for s in labels]
print("Labels", labels)

num_classes = len(labels) #Number of classes

#Selected attack
sel_attack = "genetic"
print("Selected attack: %s" % sel_attack)

#Thresholds
max_iter_thr = np.inf  #Number of iterations
max_dist_thr = 0.25    #Epsilon

#Path containing the results:
load_path = root_path + "optimization/label_shift/%s/%s/" % (sel_dataset, sel_attack)

In [ ]:
#Select the source distribution
p_y_init_idx = 0 ; p_y_init_label = "Uniform"
#p_y_init_idx = 1 ; p_y_init_label = "Tweak-2"
#p_y_init_idx = 2 ; p_y_init_label = "Tweak-4"

# Performance summary

In [ ]:
param_appendix = "_eps_%s_init_%d.npy" % (str(max_dist_thr), p_y_init_idx)
# SUCCES AND FOOLING RATE
success_optimization = np.load(load_path + "success_opt"       + param_appendix)
fooling_rates        = np.load(load_path + "fr"                + param_appendix)
opt_fooling_rate     = np.load(load_path + "opt_fr"            + param_appendix)
# KULLBACK-LEIBLER
kl_diver_obj       = np.load(load_path + "kl_obj"            + param_appendix)
# DISTANCE BASED METRICS
max_diff_obj       = np.load(load_path + "max_dif_obj"       + param_appendix)
mean_diff_obj      = np.load(load_path + "mean_dif_obj"      + param_appendix)
# CORRELATION
spearman_obj = np.load(load_path + "spearman_obj"  + param_appendix)

# SAVED ORIGINAL / ADVERSARIAL CLASSES
save_orig_classes  = np.load(load_path + "save_orig_classes" + param_appendix)
save_adv_classes   = np.load(load_path + "save_adv_classes"  + param_appendix)

# PVALUES
pval_orig_checks = np.load(load_path + "pval_orig_checks"  + param_appendix)
pval_adv_checks  = np.load(load_path + "pval_adv_checks"   + param_appendix)
checkpoints      = np.load(load_path + "checkpoints"       + param_appendix)

# INITIAL AND SET OF TARGET DISTRIBUTIONS
p_y_init    = np.load(load_path + "p_y_init"       + param_appendix)
p_y_obj_set = np.load(load_path + "p_y_obj_set"    + param_appendix)

# AUXILIARY INFORMATION
py_test_tol   = np.load(load_path + "py_tolerance" + param_appendix)


assert np.all(success_optimization==1) #Sanity check


# Number of cases in which our method managed to avoid the shift detector (for all the checks)
mask_pval_mid = np.all(pval_adv_checks>py_test_tol, axis=1)
assert len(mask_pval_mid)==len(p_y_obj_set) #Sanity check
perc_success_mid = np.sum(mask_pval_mid)/len(mask_pval_mid)*100


# Similarity metrics (considering only those cases in which our method managed to avoid the shift detector)
avg_fr       = np.mean(fooling_rates[mask_pval_mid])
avg_kl_dif   = np.mean(kl_diver_obj[mask_pval_mid])
avg_max_dif  = np.mean(max_diff_obj[mask_pval_mid])
avg_mean_dif = np.mean(mean_diff_obj[mask_pval_mid])


#Print information in tabular format
print("%s & %.2f & %.2f & %.2f &  %.2f / %.2f / %.2f \\\\"%(p_y_init_label, 
                                                            perc_success_mid,
                                                            avg_fr*100,
                                                            opt_fooling_rate*100,
                                                            avg_kl_dif, avg_max_dif, avg_mean_dif))